### 서울시 구별 주유소 데이터 분석 및 시각화

In [2]:
import numpy as np
import pandas as pd
import requests
from urllib.parse import quote
import folium, json

In [36]:
df = pd.read_csv('../daou/서울_주유소(230418)')
df.head()

,상호,주소,가격,셀프,상표,구
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구
1,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),1669,Y,SK에너지,강남구
2,삼성동주유소,서울 강남구 테헤란로 619 (삼성동),1684,Y,현대오일뱅크,강남구
3,방죽주유소,서울 강남구 밤고개로 215 (율현동),1687,Y,GS칼텍스,강남구
4,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1704,Y,SK에너지,강남구


In [37]:
df.groupby('구')

In [38]:
# 구별 집계 분석
gu_mm = df.groupby('구')['가격'].agg(['mean', 'min', 'max'])
gu_mm

,mean,min,max
구,,,
강남구,1942.294118,1623,2578
강동구,1735.000000,1669,1978
강북구,1649.916667,1629,1707
강서구,1696.818182,1628,1995
관악구,1744.714286,1685,1852
광진구,1684.538462,1654,1748
구로구,1737.200000,1645,2198
금천구,1699.090909,1665,1769
노원구,1699.428571,1629,1795


In [39]:
# 해당구에서 최대값 최소값만 구하기
# 예) 강남구
_, mi, ma = gu_mm.loc['강남구']
mi, ma = int(mi), int(ma)
mi, ma

(1623, 2578)

In [40]:
# 컬럼을 하나 추가하여 최대값인지 최솟값인지 결정해서 넣을 예정
df['mm'] = np.zeros(df.shape[0], int)
df['color'] = np.zeros(df.shape[0], str)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상호      441 non-null    object
 1   주소      441 non-null    object
 2   가격      441 non-null    int64 
 3   셀프      441 non-null    object
 4   상표      441 non-null    object
 5   구       441 non-null    object
 6   mm      441 non-null    int32 
 7   color   441 non-null    object
dtypes: int32(1), int64(1), object(6)
memory usage: 26.0+ KB


,상호,주소,가격,셀프,상표,구,mm,color
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,0,
1,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),1669,Y,SK에너지,강남구,0,
2,삼성동주유소,서울 강남구 테헤란로 619 (삼성동),1684,Y,현대오일뱅크,강남구,0,
3,방죽주유소,서울 강남구 밤고개로 215 (율현동),1687,Y,GS칼텍스,강남구,0,
4,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1704,Y,SK에너지,강남구,0,


In [41]:
# mm컬럼에 최소값이 같으면 1을 넣고 최대값과 같으면 2를 넣음것
# 칼라도 추가, 최소는 초록, 최대는 빨강

for gu in gu_mm.index:
    _, mi, ma = gu_mm.loc[gu]
    mi, ma = int(mi), int(ma)
    for i in df.index:
        if df.가격[i] == mi and df.구[i] == gu:
            df.mm[i] = 1
            df.color[i] = 'green'
        if df.가격[i] == ma and df.구[i] == gu:
            df.mm[i] = 2
            df.color[i] = 'red'
            

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_7488\2850588523.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.mm[i] = 1
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_7488\2850588523.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.color[i] = 'green'
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_7488\2850588523.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.mm[i] = 2
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_7488\

In [42]:
df.head()

,상호,주소,가격,셀프,상표,구,mm,color
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,1,green
1,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),1669,Y,SK에너지,강남구,0,
2,삼성동주유소,서울 강남구 테헤란로 619 (삼성동),1684,Y,현대오일뱅크,강남구,0,
3,방죽주유소,서울 강남구 밤고개로 215 (율현동),1687,Y,GS칼텍스,강남구,0,
4,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1704,Y,SK에너지,강남구,0,


In [43]:
# mm 컬럼에 데이터가 있는 행만 추출
df1 = df[df.mm !=0]
df1.shape
df1

,상호,주소,가격,셀프,상표,구,mm,color
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,1,green
33,(주)만정에너지 삼보주유소,서울 강남구 봉은사로 433 (삼성동),2578,N,GS칼텍스,강남구,2,red
34,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1669,Y,현대오일뱅크,강동구,1,green
46,광성주유소,서울 강동구 올림픽로 673 (천호동),1978,N,S-OIL,강동구,2,red
47,SK에너지(주) 매일주유소,서울 강북구 삼양로 217 (미아동),1629,Y,SK에너지,강북구,1,green
...,...,...,...,...,...,...,...,...
417,(주)대양씨앤씨 사직주유소,서울 종로구 사직로 65 (사직동),2240,N,GS칼텍스,종로구,2,red
418,다산주유소,서울 중구 다산로 233,1649,Y,S-OIL,중구,1,green
427,서남주유소,서울 중구 통일로 30,2753,N,SK에너지,중구,2,red
428,(주)자연에너지 대창주유소,서울 중랑구 동일로 636 (면목동),1635,Y,현대오일뱅크,중랑구,1,green


지도에 시각화
1. KAKAO API로 위경도값 추출

In [52]:
# 1 .. KAKAO API로 위경도값 추출
# api 로드
with open(r'C:\Users\tjoeun\Desktop\수업자료\수업_실습\API\카카오API.txt') as file:
    kakao_key = file.read()

In [53]:
# 주소 호출 및 삽입
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
lat_list, lon_list= [], []

for i in df1.index:
    temp = quote(df1.주소[i])
    url = f'{search_url}?query={temp}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
    try:
        lon_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df1.상호[i], " 의 위경도 정보를 찾을 수 없습니다")
        lon_list.append(None)
        lat_list.append(None)

df1['위도'] = lat_list
df1['경도'] = lon_list

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_7488\3835906461.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['위도'] = lat_list
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_7488\3835906461.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['경도'] = lon_list


In [54]:
df1.head()

,상호,주소,가격,셀프,상표,구,mm,color,위도,경도
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,1,green,37.467272,127.118743
33,(주)만정에너지 삼보주유소,서울 강남구 봉은사로 433 (삼성동),2578,N,GS칼텍스,강남구,2,red,37.511551,127.047115
34,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1669,Y,현대오일뱅크,강동구,1,green,37.536417,127.149372
46,광성주유소,서울 강동구 올림픽로 673 (천호동),1978,N,S-OIL,강동구,2,red,37.541577,127.124242
47,SK에너지(주) 매일주유소,서울 강북구 삼양로 217 (미아동),1629,Y,SK에너지,강북구,1,green,37.622765,127.019086


In [68]:
# .. 지도에 정보 표시
import folium

# .. 단계구분도용 geo_stn
#filename =  '../../mapdata/skorea_municipalities_geo_simple.json'
#geo_stn = json.load(open(filename, encoding='utf-8'))


In [ ]:

map = folium.Map(location=[df1.위도.mean(),df1.경도.mean()], zoom_start=12)

for i in df1.index:
    folium.Marker(location=[df1.위도[i],df1.경도[i]],
                  popup=folium.Popup(df1.주소[i], max_width=200),
                  tooltip = f'{df1.상호[i]}<br>휘발유:{df1.가격[i]:,d}원',
                  icon=folium.Icon(color='green' if df1.mm[i] == 1 else 'red',
                                    icon='glyphicon glyphicon-thumbs-up' if df1.mm[i] == 1 else 'glyphicon glyphicon-thumbs-down')).add_to(map)
title = '<h3 align="center" style="font-size:20"> 구별 최대/최소 가격 주유소</h3>'
map.get_root().html.add_child(folium.Element(title))
map


단계구분도 시각화 (과제, 주유소값 평균으로)
* statmen toner로 